In [ ]:
import numpy as np
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()

### Problem statement

<img src="joint-tuning-problem.jpeg" alt="brainstorm" width="300"/>

### Placeholder values

In [ ]:
a = 1.9; b = 1.7
C = 62*np.pi/180
# by measurement of the model)

t = 30*np.pi/180  # 0 <= t <= C
# example

### One triangle of the joint

Let's look at quadrant 1 -- the rest are just mirror images in the axes.<br>
Constraints on the triangle:
- One vertex at (0,0,0) (vertex C)
- One edge in the YZ plane (edge a)
- One edge in the XZ plane (edge b)

Degrees of freedom / parameters that characterize it:
- Length of a
- Length of b
- cos(a, b) = cos(C)

There is an extra degree of freedom along which this triangle is free to move.<br>
Looking at the big picture model, it makes sense to characterize this parameter by<br>
$t = \theta/2 = (\textrm{the angle made by b with Z})$ ... For our purposes, $0 \leq t \leq C$

In [ ]:
pointC = np.array([0, 0, 0])

From this, the corner A (opposite side a) has coordinates:<br>
$A = (b*\sin(t), 0, b*\cos(t))$

In [ ]:
pointA = np.array([b*np.sin(t), 0, b*np.cos(t)])

The corner B (opposite side b) has coordinates:<br>
$B = (0, yy, zz)$  (say) such that<br>
$yy^{2} + zz^{2} = a^{2}$ &nbsp; &nbsp; -- (length of the edge)<br>
$zz * b * \cos(t) = a*b*\cos(C)$ &nbsp; &nbsp; -- (dot product between the two vectors)

Thus,<br>
$B = (0, a*\sqrt{1 - \frac{\cos^2(C)}{\cos^2(t)}}, a*\frac{\cos(C)}{\cos(t)})$

For shorthand, say B makes an angle $s$ with the Z axis, and hence has coordinates<br>
$B = (0, a*\sin(s), a*\cos(s))$ where $s = \cos^{-1}\left(\frac{cos(C)}{cos(t)}\right)$

In [ ]:
s = np.arccos(np.cos(C)/np.cos(t))
pointB = np.array([0, a*np.sin(s), a*np.cos(s)])

Thus,

In [ ]:
quad1_vertices = np.array([pointA, pointB, pointC])

### Now, we compute $\phi$

We'll drop a perpendicular from B (say BH) onto the line AC.<br>
The angle that BH has with the XZ plane = $\phi/2$

Let $H = (h*\sin(t), 0, h*\cos(t))$  (since H is on line AC)

Constraint : $BH \perp AC$<br>
$\Rightarrow (0, a*\sin(s), a*\cos(s)) - (h*\sin(t), 0, h*\cos(t)) \perp (b*\sin(t), 0, b*\cos(t))$<br>
$\Rightarrow (-h*\sin(t), a*\sin(s), a*\cos(s) - h*\cos(t)) \perp (b*\sin(t), 0, b*\cos(t))$<br>
$\Rightarrow -h*\sin(t) * b*\sin(t) + 0 + (a*\cos(s) - h*\cos(t)) * b*\cos(t) = 0$<br>
$\Rightarrow -h*b + a*\cos(s) * b*\cos(t) = 0$<br>
$\Rightarrow h = a*\cos(s)*\cos(t)$<br>

Thus,

In [ ]:
h = a*np.cos(s)*np.cos(t)
pointH = np.array([h*np.sin(t), 0, h*np.cos(t)])

Given any vector (x, y, z), the angle that it makes with the XZ plane
$= \tan^{-1}\left(\frac{y}{\sqrt(x^2 + z^2)}\right)$

In [ ]:
BH = pointB - pointH

In [ ]:
phi = 2*np.arctan(BH[1]/np.linalg.norm([BH[0], BH[2]]))

#### Important: This does not depend on a & b. We can omit out those factors.

### Making it all concise...

In [ ]:
# The only constant that we require
C = 62*np.pi/180

In [ ]:
def phi_vs_theta(theta):
    t = theta/2
    s = np.arccos(np.cos(C)/np.cos(t))
    pointB = np.array([0, np.sin(s), np.cos(s)])
    h = np.cos(s)*np.cos(t)
    pointH = np.array([h*np.sin(t), 0, h*np.cos(t)])
    BH = pointB - pointH
    phi = 2*np.arctan(BH[1]/np.linalg.norm([BH[0], BH[2]]))
    return phi

Now, let's plot this

In [ ]:
thetas = np.arange(0, 2*C, 0.01)
pvtv = np.vectorize(phi_vs_theta)
phis = pvtv(thetas)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(thetas*180/np.pi, phis*180/np.pi)
plt.title("$\phi$ v/s $\Theta$")

In [ ]:
plt.plot(phis*180/np.pi, thetas*180/np.pi)
plt.title("$\Theta$ v/s $\phi$ (as in the diagram)")

### Does $\phi + 2*\theta$ ever go to 360?

In [ ]:
plt.plot(thetas*180/np.pi, (phis + 2*thetas)*180/np.pi)
plt.title("$\phi + 2*\Theta$")

nope!

### What about a larger C?

In [ ]:
C = 80*np.pi/180
pvtv = np.vectorize(phi_vs_theta)
phis = pvtv(thetas)

In [ ]:
plt.plot(thetas*180/np.pi, (phis + 2*thetas)*180/np.pi)
plt.title("$\phi + 2*\Theta$")

At roughly C=73.5 degrees, we start to see a point touching 360. Thus, we need a C > this.

In [ ]:
np.pi*2

In [ ]:
theta = 158.5445*np.pi/180
phi_vs_theta(theta) + 2*theta

In [ ]:
theta = 102.746*np.pi/180
phi_vs_theta(theta) + 2*theta

In [ ]:
360 - 2*158.5445, 360 - 2*102.746

In [ ]:
102.746*4 + 154.508*2